## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file.
city_df = pd.read_csv("WeatherPy_database.csv")
city_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Denpasar,ID,-8.6500,115.2167,83.86,74,20,2.30,few clouds
1,1,Jamestown,US,42.0970,-79.2353,40.59,50,100,1.99,overcast clouds
2,2,Bluff,NZ,-46.6000,168.3333,57.47,90,100,18.32,light rain
3,3,Athabasca,CA,54.7169,-113.2854,-27.42,100,23,5.55,few clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,81.95,94,100,29.51,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_df.loc[(city_df['Max Temp']<= max_temp) & (city_df['Max Temp']>= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Denpasar,ID,-8.6500,115.2167,83.86,74,20,2.30,few clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,81.95,94,100,29.51,moderate rain
7,7,Rikitea,PF,-23.1203,-134.9692,77.49,71,49,6.64,scattered clouds
8,8,Dondo,MZ,-19.6094,34.7431,85.95,59,31,10.09,scattered clouds
11,11,Busselton,AU,-33.6500,115.3333,78.04,33,99,13.20,overcast clouds
15,15,Necochea,AR,-38.5473,-58.7368,86.04,42,99,9.17,overcast clouds
24,24,Carnot,CF,4.9409,15.8665,95.54,12,93,2.48,overcast clouds
25,25,Hermanus,ZA,-34.4187,19.2345,79.81,70,28,24.58,scattered clouds
26,26,Vaini,TO,-21.2000,-175.2000,77.16,88,20,3.44,few clouds
28,28,Chuy,UY,-33.6971,-53.4616,79.93,50,0,6.76,clear sky


In [5]:
# 4a. Determine if there are any empty rows. & show info
preferred_cities_df.isnull().sum()
#preferred_cities_df.info()

City_ID                0
City                   0
Country                7
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Denpasar,ID,83.86,few clouds,-8.6500,115.2167,
4,Saint-Philippe,RE,81.95,moderate rain,-21.3585,55.7679,
7,Rikitea,PF,77.49,scattered clouds,-23.1203,-134.9692,
8,Dondo,MZ,85.95,scattered clouds,-19.6094,34.7431,
11,Busselton,AU,78.04,overcast clouds,-33.6500,115.3333,
15,Necochea,AR,86.04,overcast clouds,-38.5473,-58.7368,
24,Carnot,CF,95.54,overcast clouds,4.9409,15.8665,
25,Hermanus,ZA,79.81,scattered clouds,-34.4187,19.2345,
26,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
28,Chuy,UY,79.93,clear sky,-33.6971,-53.4616,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

    try:
         hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
amend_hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
amend_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Denpasar,ID,83.86,few clouds,-8.6500,115.2167,ASTON Denpasar Hotel & Convention Center
4,Saint-Philippe,RE,81.95,moderate rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
7,Rikitea,PF,77.49,scattered clouds,-23.1203,-134.9692,People ThankYou
8,Dondo,MZ,85.95,scattered clouds,-19.6094,34.7431,Arco Iris
11,Busselton,AU,78.04,overcast clouds,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
684,Acarau,BR,85.08,overcast clouds,-2.8856,-40.1200,castelo pizzaria
686,Caravelas,BR,88.45,light rain,-17.7125,-39.2481,Pousada dos Navegantes
690,Yondo,CO,77.99,few clouds,7.0062,-73.9097,estacion sayab sede la felicidad
697,San Luis De Gaceno,CO,88.54,overcast clouds,4.8205,-73.1685,Hotel Buenos Aires


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search.csv"

# 8b. Export the City_Data into a csv
amend_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} &#176;F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in amend_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = amend_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))